# [Module 2.3] 세이지 메이커 사용자 도커 이미지로 인퍼런스

## 필수 사항
- 이 노트북은 sagemaker_inference_container/container-inference/ 의 아래 두개의 노트북을 먼저 실행해야 합니다.
    - 1.1.Build_Docker.ipynb
    - 2.1.Package_Model_Artifact.ipynb
본 워크샵의 모든 노트북은 `conda_python3` 추가 패키지를 설치하고 모두 이 커널 에서 작업 합니다.


## 이 노트북은 아래와 같은 작업을 합니다.

- 1. 환경 셋업
- 2. 로컬 모드에서 배포 및 추론 테스트
- 3. 세이지 메이커 (호스트 모드)에서 배포 및 추론 테스트
- 4. 세이지 메이커 신규 모델로 앤드포인트 업데이트
- 5. 신규 엔드포인트 생성
- 6. 신규 앤드포인트 생성 후에 Bake Time (예: 5분) 지난 후에 기존 엔드포인트 삭제
- 7. 마지막 신규 엔드포인트 리소스 제거


---    
    

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

이전 노트북에서 인퍼런스 테스트를 완료한 티펙트를 가져옵니다.
- inference_docker_image: 사용자 정의 도커 이미지
- byom_artifact: 추론 코드를 포함한 모델 아티펙트

In [2]:
%store -r inference_docker_image
# %store -r artifact_path
%store -r byom_artifact
%store -r bucket
%store -r prefix

In [3]:
print("inference_docker_image: \n", inference_docker_image)
print("byom_artifact: \n", byom_artifact)

inference_docker_image: 
 057716757052.dkr.ecr.us-east-1.amazonaws.com/ncf-sagemaker-inference
byom_artifact: 
 s3://sagemaker-us-east-1-057716757052/sm2fraud/new2train2model/model.tar.gz


In [4]:
import sagemaker

role = sagemaker.get_execution_role()

# 2. 로컬 모드에서 배포 및 추론 테스트


## Python SDK를 통한 세이지 메이커 모델 생성
- 로컬 엔드포인트에 적재될 세이지 메이커 모델을 생성홥니다.
- 세이지 메이커 모델은 크게 아래 두가지 요소가 있습니다.
    - 인퍼런스 이미지 컨테이너 위치
    - 모델 아티펙트 위치

In [5]:
from datetime import datetime
suffix = f"{datetime.today().strftime('%Y-%m-%d-%H-%M-%S')}"

# 로컬 노트북 인스턴스에서 사용할 로컬 세션
local_session = sagemaker.local.LocalSession()
local_session.config = {'local' : {'local_code':True}}
instance_type = 'local_gpu'
local_model_name = 'sm-local-model-' + str(suffix)
local_endpoint_name = 'local-endpoint-' + str(suffix)
print("local_endpoint_name: ", local_endpoint_name)

local_endpoint_name:  local-endpoint-2022-10-16-13-24-46


### 로컬 세이지 메이커 모델 정의

In [6]:
from sagemaker.model import Model


def create_sm_model(ecr_image, model_artifact,model_name,  role, session):
    model = Model(
                name = model_name,
                image_uri = ecr_image,
                model_data = model_artifact,
                role=role,
                sagemaker_session= session,
                 )
    print("model: ", model)
    return model

local_model = create_sm_model(inference_docker_image, byom_artifact, 
                              local_model_name, role, 
                              local_session)    

model:  <sagemaker.model.Model object at 0x7f1aebc57c10>


### 로컬 엔드포인트 생성

- instance_type=='local' 시에는 최초 실행시에 인퍼런스 이미지를 다운로드로 하는 약 3분 걸리고, 이후에는 바로 실행이 됩니다.

In [7]:
%%time

local_predictor = local_model.deploy(initial_instance_count=1, instance_type=instance_type,
                         endpoint_name = local_endpoint_name,                                   
                        )

Attaching to 0rpq12cibv-algo-1-4gkb0
0rpq12cibv-algo-1-4gkb0 | Collecting nvidia-ml-py3==7.352
0rpq12cibv-algo-1-4gkb0 |   Downloading nvidia-ml-py3-7.352.0.tar.gz (19 kB)
0rpq12cibv-algo-1-4gkb0 |   Preparing metadata (setup.py) ... done
0rpq12cibv-algo-1-4gkb0 | Collecting pandas==0.24.2
0rpq12cibv-algo-1-4gkb0 |   Downloading pandas-0.24.2-cp36-cp36m-manylinux1_x86_64.whl (10.1 MB)
     |████████████████████████████████| 10.1 MB 38.9 MB/s            
0rpq12cibv-algo-1-4gkb0 | Collecting numpy==1.16.6
0rpq12cibv-algo-1-4gkb0 |   Downloading numpy-1.16.6-cp36-cp36m-manylinux1_x86_64.whl (17.4 MB)
     |████████████████████████████████| 17.4 MB 54.4 MB/s            
0rpq12cibv-algo-1-4gkb0 | Requirement already satisfied: torch==1.8.1 in /opt/conda/lib/python3.6/site-packages (from -r /opt/ml/model/code/requirements.txt (line 4)) (1.8.1)
0rpq12cibv-algo-1-4gkb0 | Collecting gensim==3.7.1
0rpq12cibv-algo-1-4gkb0 |   Downloading gensim-3.7.1-cp36-cp36m-manylinux1_x86_64.whl (24.2 MB)
   

## 테스트 데이터 세트 로딩
- 로컬에서 저장된 데이터를 가져와서 데이터를 변환 합니다.
- batch_size 만큼 데이터를 로딩하는 데이터 로더를 정의 합니다.

In [8]:
import data_utils 
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

##  추론을 위한  데이터 세트 로딩
- 전부 데이터를 로딩할 필요가 없지만, 여기서는 기존에 사용한 함수를 이용하기 위해서 전체 데이터를 로드 합니다. 
    - 실제 데이터로 구현시에는 따로이 로드 함수를 사용하시기를 권장 합니다.


In [9]:
class Params:
    def __init__(self):
        # self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        # self.lr = 0.001
        self.top_k = 10
        self.out = True
        # self.gpu = "0"
                        
args = Params()
print("# of batch_size: ", args.batch_size)


import torch.utils.data as data

test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)

test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



# of batch_size:  256


## 추론할 Paylaod 하나를 생성

In [10]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}
print("payload: ", payload)


payload:  {'user': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'item': [25, 1064, 174, 2791, 3373, 269, 2678, 1902, 3641, 1216, 915, 3672, 2803, 2344, 986, 3217, 2824, 2598, 464, 2340, 1952, 1855, 1353, 1547, 3487, 3293, 1541, 2414, 2728, 340, 1421, 1963, 2545, 972, 487, 3463, 2727, 1135, 3135, 128, 175, 2423, 1974, 2515, 3278, 3079, 1527, 2182, 1018, 2800, 1830, 1539, 617, 247, 3448, 1699, 1420, 2487, 198, 811, 1010, 1423, 2840, 1770, 881, 1913, 1803, 1734, 3326, 1617, 224, 3352, 1869, 1182, 1331, 336, 2517, 1721, 3512, 3656, 273, 1026, 1991, 2190, 998, 3386, 3369, 185, 2822, 864, 2854, 3067, 58, 2551, 2333, 2688, 3703, 1300, 1924, 3118]}


## 엔드 포인트 추론
- Boto3 invoke_endpoint() 로 추론

In [11]:
import boto3

if instance_type == 'local_gpu':
    runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
    endpoint_name = local_endpoint_name
else:
    runtime_client = boto3.Session().client('sagemaker-runtime')



### [중요] JSON type 의 payload 를 String 으로 직렬화 해서 제공 함.
```python
payload_dump = json.dumps(payload)
```

In [12]:
import json, time
from inference_utils import invoke_endpoint
payload_dump = json.dumps(payload)

start_time = time.time()
result = invoke_endpoint(runtime_client, endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)

0rpq12cibv-algo-1-4gkb0 | 2022-10-16 13:25:20,406 [INFO ] W-9000-model_1-stdout MODEL_LOG - #### input_fn starting ######
0rpq12cibv-algo-1-4gkb0 | 2022-10-16 13:25:20,406 [INFO ] W-9000-model_1-stdout MODEL_LOG - content_type: application/json
0rpq12cibv-algo-1-4gkb0 | 2022-10-16 13:25:20,406 [INFO ] W-9000-model_1-stdout MODEL_LOG - #### type of input data: <class 'str'>
0rpq12cibv-algo-1-4gkb0 | 2022-10-16 13:25:20,406 [INFO ] W-9000-model_1-stdout MODEL_LOG - #### predict_fn starting ######
0rpq12cibv-algo-1-4gkb0 | 2022-10-16 13:25:20,407 [INFO ] W-9000-model_1-stdout MODEL_LOG - #### type of input data: <class 'list'>
0rpq12cibv-algo-1-4gkb0 | 2022-10-16 13:25:20,823 [INFO ] W-9000-model_1-stdout MODEL_LOG - recommends:  [273, 174, 128, 25, 175, 1331, 1182, 1064, 464, 1539]
0rpq12cibv-algo-1-4gkb0 | 2022-10-16 13:25:20,823 [INFO ] W-9000-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:417.54|#ModelName:model,Level:Model|#hostname:9636777b54bf,requestID:c41fb1ec-5cce-4c

## 로컬 엔드 포인트 삭제
- 기존에 생성한 세이지 메이커 모델, 앤드포인트 컨피그, 앤드포인트 삭제

In [13]:
from inference_utils import delete_endpoint

client = sagemaker.local.LocalSagemakerClient()
delete_endpoint(client, endpoint_name)

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: sm-local-model-2022-10-16-13-24-46
--- Deleted endpoint: local-endpoint-2022-10-16-13-24-46
--- Deleted endpoint_config: local-endpoint-2022-10-16-13-24-46


# 3. 세이지 메이커 (호스트 모드)에서 배포 및 추론 테스트
- 위의 단계에 (로컬 앤드포인트)에서 추론 테스트가 완료 되었기에, 프러덕션을 가정하고 배포 및 추론을 합니다.
- [중요] 기존의 SageMaker Python SDK 대신에 Boto3 를 사용합니다.
    -  참조: [Boto3 API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.describe_endpoint_config)

## SageMaker 호스트 세션 및 변수 설정

In [14]:
import boto3
# SM 세션에서 사용 리소스 제거에 사용
sm_client = boto3.client('sagemaker')
sm_runtime_client = boto3.Session().client('sagemaker-runtime')

sm_model_name = 'sm-ncf-model-' + str(suffix)
endpoint_name = 'sm-new-ncf-endpont-' + str(suffix)
print("sm_model_name: ", sm_model_name)
print("inference_image: ", inference_docker_image)
print("model_artifact: ", byom_artifact)
print("endpoint_name: ", endpoint_name)

sm_model_name:  sm-ncf-model-2022-10-16-13-24-46
inference_image:  057716757052.dkr.ecr.us-east-1.amazonaws.com/ncf-sagemaker-inference
model_artifact:  s3://sagemaker-us-east-1-057716757052/sm2fraud/new2train2model/model.tar.gz
endpoint_name:  sm-new-ncf-endpont-2022-10-16-13-24-46


## 세이지 메이커 모델 생성
- 세이지 메이커에 Model이 존재하지 않으면, 모델을 생성 합니다.

In [15]:
from inference_utils import create_sm_model                

create_model_api_response = create_sm_model(sm_client, sm_model_name, 
                                            inference_docker_image, 
                                            byom_artifact, role)

create_model API response: 
 {'ModelArn': 'arn:aws:sagemaker:us-east-1:057716757052:model/sm-ncf-model-2022-10-16-13-24-46', 'ResponseMetadata': {'RequestId': 'b3e599f4-4622-47a2-b880-a702a541786b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b3e599f4-4622-47a2-b880-a702a541786b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '94', 'date': 'Sun, 16 Oct 2022 13:25:31 GMT'}, 'RetryAttempts': 0}}


## 세이지 메이커 앤드포인트 컨피그 생성
- endpoint_config  를 생성합니다.

In [16]:
from inference_utils import create_endpoint_config

instance_type = 'ml.g4dn.xlarge'
sm_endconfig_name = create_endpoint_config(sm_client=sm_client, 
                                           instance_type=instance_type, 
                                           sm_model_name =sm_model_name, 
                                           verbose=False )
    
print("sm_endconfig_name: ", sm_endconfig_name )

sm-ncf-model-2022-10-16-13-24-46-config is created
sm_endconfig_name:  sm-ncf-model-2022-10-16-13-24-46-config


## 세이지 메이커 앤드포인트 생성
- 약 8분 정도 소요 됩니다. 

In [17]:
from inference_utils import create_sm_endpoint

create_sm_endpoint(sm_client=sm_client, instance_type= instance_type, 
                   endpoint_config_name =sm_endconfig_name, 
                   endpoint_name = endpoint_name, verbose=True )
    

existing_configs: 
 [{'EndpointConfigName': 'sm-ncf-model-2022-10-16-13-24-46-config', 'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:057716757052:endpoint-config/sm-ncf-model-2022-10-16-13-24-46-config', 'CreationTime': datetime.datetime(2022, 10, 16, 13, 25, 32, 180000, tzinfo=tzlocal())}]
existing_endpoints: 
 []
Creating endpoint
Endpoint status is creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: InService


## SageMaker 엔드포인트 추론 테스트
- 신규로 생성한 앤드포인트가 추론이 달 되는지를 확인하기 위함

In [18]:
import boto3

if instance_type == 'local_gpu':
    runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
    endpoint_name = local_endpoint_name
else:
    runtime_client = boto3.Session().client('sagemaker-runtime')


import json, time
from inference_utils import invoke_endpoint
payload_dump = json.dumps(payload)

start_time = time.time()
result = invoke_endpoint(runtime_client, endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)


--- 1.4708614349365234 seconds ---
result:  ['[273, 174, 128, 25, 175, 1331, 1182, 1064, 464, 1539]']


# 4. 세이지 메이커 신규 모델로 앤드포인트 업데이트

### current_endpoint_name objects
- 기존의 앤드포인트의 오브젝트 정보 확인
    - 앤드포인트 이름
    - 앤드포인트 컨피그 이름
    - 세이지 메이커 모델 이름

In [19]:
from inference_utils import show_inference_objects

In [20]:
show_inference_objects(sm_client, endpoint_name)    

endpoint_name: 
 sm-new-ncf-endpont-2022-10-16-13-24-46
endpoint_config: 
 sm-ncf-model-2022-10-16-13-24-46-config
model_name: 
 sm-ncf-model-2022-10-16-13-24-46


### new_endpoint_name objects
- 위와 동일하게 신규 생성 앤드포인트의 오브젝트 이름을 확인

## 4.1. 신규 세이지 메이커 모델 생성

In [21]:
import boto3
new_suffix = f"{datetime.today().strftime('%Y-%m-%d-%H-%M-%S')}"

sm_new_model_name = 'sm-ncf-model-' + str(new_suffix)

byom_new_artifact = byom_artifact
print("sm_new_model_name: ", sm_new_model_name)
print("byom_new_artifact: ", byom_new_artifact)

create_model_api_response = create_sm_model(sm_client, sm_new_model_name, 
                                            inference_docker_image, byom_new_artifact, role)

sm_new_model_name:  sm-ncf-model-2022-10-16-13-30-35
byom_new_artifact:  s3://sagemaker-us-east-1-057716757052/sm2fraud/new2train2model/model.tar.gz
create_model API response: 
 {'ModelArn': 'arn:aws:sagemaker:us-east-1:057716757052:model/sm-ncf-model-2022-10-16-13-30-35', 'ResponseMetadata': {'RequestId': 'f3f3250b-431f-413f-be85-581b2032b1b2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f3f3250b-431f-413f-be85-581b2032b1b2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '94', 'date': 'Sun, 16 Oct 2022 13:30:36 GMT'}, 'RetryAttempts': 0}}


## 4.2. 신규 엔드포인트 컨피그 생성

In [22]:
sm_new_endconfig_name = create_endpoint_config(sm_client=sm_client, 
                                           instance_type=instance_type, 
                                           sm_model_name =sm_new_model_name, 
                                           verbose=False )
    
print("sm_new_endconfig_name: ", sm_new_endconfig_name )

sm-ncf-model-2022-10-16-13-30-35-config is created
sm_new_endconfig_name:  sm-ncf-model-2022-10-16-13-30-35-config


## 4.3 앤드포인트 업데이트
- 아래의 업데이트는 약 8분 정도가 소요 됩니다.
- 내부적으로 EC2 인스턴스 생성하고 배포하는 단계를 가집니다.
- 8분 동안 업데이트시에도 기존의 앤드포인트로 서비스가 가능합니다.
- 업데이트 이후에는 업데이트된 앤드포인트에서 추론이 됩니다.

In [23]:
from inference_utils import update_sm_endpoint

response = update_sm_endpoint(sm_client, endpoint_name, sm_new_endconfig_name)
response

endpoint_name: 
 sm-new-ncf-endpont-2022-10-16-13-24-46
new_endpoint_config_name:  sm-ncf-model-2022-10-16-13-30-35-config
current_endpoint_config_name:  sm-ncf-model-2022-10-16-13-24-46-config
Swapping is done
changed_endpoint_config_name: 
 sm-ncf-model-2022-10-16-13-24-46-config


{'EndpointArn': 'arn:aws:sagemaker:us-east-1:057716757052:endpoint/sm-new-ncf-endpont-2022-10-16-13-24-46',
 'ResponseMetadata': {'RequestId': '8df019ba-0fc5-4ff7-91ff-b93fd2a8986a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8df019ba-0fc5-4ff7-91ff-b93fd2a8986a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '106',
   'date': 'Sun, 16 Oct 2022 13:30:36 GMT'},
  'RetryAttempts': 0}}

In [26]:
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']

while status == 'Updating':
    print("endpoint status is ", status)    
    time.sleep(30)
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = response['EndpointStatus']




In [28]:
show_inference_objects(sm_client, endpoint_name)    

endpoint_name: 
 sm-new-ncf-endpont-2022-10-16-13-24-46
endpoint_config: 
 sm-ncf-model-2022-10-16-13-30-35-config
model_name: 
 sm-ncf-model-2022-10-16-13-30-35


# 5. 신규 엔드포인트 생성

In [29]:
new_endpoint_name = 'sm-new-ncf-endpont-' + str(new_suffix)
print("sm_new_model_name: ", sm_new_model_name)
print("sm_new_endconfig_name: ", sm_new_endconfig_name)
print("byom_new_artifact: ", byom_new_artifact)
print("new_endpoint_name: ", new_endpoint_name)

sm_new_model_name:  sm-ncf-model-2022-10-16-13-30-35
sm_new_endconfig_name:  sm-ncf-model-2022-10-16-13-30-35-config
byom_new_artifact:  s3://sagemaker-us-east-1-057716757052/sm2fraud/new2train2model/model.tar.gz
new_endpoint_name:  sm-new-ncf-endpont-2022-10-16-13-30-35


In [30]:
create_sm_endpoint(sm_client=sm_client, instance_type= instance_type, 
                   endpoint_config_name =sm_new_endconfig_name, 
                   endpoint_name = new_endpoint_name, verbose=True )



existing_configs: 
 [{'EndpointConfigName': 'sm-ncf-model-2022-10-16-13-30-35-config', 'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:057716757052:endpoint-config/sm-ncf-model-2022-10-16-13-30-35-config', 'CreationTime': datetime.datetime(2022, 10, 16, 13, 30, 36, 596000, tzinfo=tzlocal())}]
existing_endpoints: 
 []
Creating endpoint
Endpoint status is creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: Creating
Endpoint status: InService


# 6. 신규 앤드포인트 생성 후에 Bake Time (예: 5분) 지난 후에 기존 엔드포인트 삭제

In [31]:
# time.sleep(300)

In [32]:
from inference_utils import delete_endpoint_detail
delete_endpoint_detail(client = sm_client, 
                endpoint_name = endpoint_name,
                is_delete=True, 
                is_del_model=False, 
                is_del_endconfig=False,
                is_del_endpoint=True)




model_name: 
 sm-ncf-model-2022-10-16-13-30-35
EndpointConfigName: 
 sm-ncf-model-2022-10-16-13-30-35-config
endpoint_name: 
 sm-new-ncf-endpont-2022-10-16-13-24-46
--- Deleted endpoint_config: sm-ncf-model-2022-10-16-13-30-35-config


# 7. 마지막 신규 엔드포인트 리소스 제거
- 첫번째는 실제 리소스를 제제거하지 않고, 관련 오브젝트만을 보여 줍니다.
- 두번째는 신규 생성한 세이지메이커 모델, 엔드포인트 컨피그는 지우지 않고, 앤프포인트만 삭제 합니다.
    - 이유는 업데이트된 앤드포인트에서 사용하기 때문입니다.

In [33]:
from inference_utils import delete_endpoint

In [34]:
from inference_utils import delete_endpoint_detail
delete_endpoint_detail(client = sm_client, 
                endpoint_name = new_endpoint_name,
                is_delete=True, 
                is_del_model=True, 
                is_del_endconfig=True,
                is_del_endpoint=True)



model_name: 
 sm-ncf-model-2022-10-16-13-30-35
EndpointConfigName: 
 sm-ncf-model-2022-10-16-13-30-35-config
endpoint_name: 
 sm-new-ncf-endpont-2022-10-16-13-30-35
--- Deleted endpoint: sm-new-ncf-endpont-2022-10-16-13-30-35
--- Deleted model: sm-ncf-model-2022-10-16-13-30-35
--- Deleted endpoint_config: sm-ncf-model-2022-10-16-13-30-35-config
